In [1]:
import pandas as pd
import html5lib
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

In [2]:
dataframes=[]
for k in tqdm(range(2001,2019)):
    for i in range(1,13):
        df = pd.read_html('./data/'+str(k)+'Y_'+str(i)+'M.xls',encoding='utf-8',header=0)
        dataframes.append(df[0])

100%|██████████| 18/18 [00:40<00:00,  4.45s/it]


In [3]:
df_nans=[]
for df in dataframes:
    df_nan=df[df.isna().any(1)]
    df_nan=df_nan.dropna(axis='columns')
    # 2007/12 2013/12 change
    df_nan.rename(columns={'類型代號':"基金名稱","基金統編":"基金規模","基金規模":"本月定時定額扣款筆數","單位淨值":"本月定時定額扣款人數","本月定時定額扣款筆數":"本月定時定額扣款金額"})
    df_nans.append(df_nan)

In [26]:
df_corrects=[]
df_filter = []
# for df in dataframes:
#     df_correct = df[~df.isna().any(1)]
#     df_corrects.append(df_correct)
for df in tqdm(dataframes):
    df.rename(columns={"基金規模(台幣)":"基金規模","單位淨值(台幣)":"單位淨值","本月定時定額扣款金額(台幣)":"本月定時定額扣款金額"}, inplace=True)
    df_correct = df[~df.isna().any(1)]
    df_filter = []
    for name in df_correct['基金名稱']:
        if ("中國" in name or "俄羅斯" in name or "巴西" in name or "印度" in name or "醫療" in name or "生技" in name) :
            if("中國信託" in name):
                continue
            filter1 = (df_correct['類型代號'] == "AA2")
            filter2 = (df_correct['類型代號'] == "2")
            filter3 = (df_correct['基金名稱'] == name)
            df_filter.append(df_correct[((filter1 | filter2) & filter3)])
    df_corrects.append(pd.concat(df_filter))
df_corrects[0]

100%|██████████| 216/216 [00:24<00:00,  1.86it/s]


,類型代號,基金統編,基金名稱,基金規模,單位淨值,本月定時定額扣款筆數,本月定時定額扣款人數,本月定時定額扣款金額
86,2,97990289,元大中國基金,436898521,6.54,8,8.0,63000.0


In [27]:
def plot_image(id):
    navs=[]
    money=[]
    num=[]
    people=[]
    scales=[]
    id = str(id)
    id = id.zfill(8)
    for i in range(0,2*12):
        try:
            navs.append(df_corrects[i][df_corrects[i]['基金統編']==id]['單位淨值'].values[0])
            money.append(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款金額'].values[0])
            num.append(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款筆數'].values[0])
            people.append(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款筆數'].values[0])
            scales.append(df_corrects[i][df_corrects[i]['基金統編']==id]['基金規模'].values[0])
        except:
            pass
    plt.plot(navs)
    plt.title('Unit net value')
    plt.ylabel('Value')
    plt.xlabel('Months')
    plt.show()
    plt.title('SIP value')
    plt.plot(money)
    plt.ylabel('Value')
    plt.xlabel('Months')
    plt.show()
    plt.title('SIP Number of records')
    plt.plot(num)
    plt.ylabel('Value')
    plt.xlabel('Months')
    plt.show()
    plt.title('SIP Number of people')
    plt.plot(people)
    plt.ylabel('Value')
    plt.xlabel('Months')
    plt.show()
    plt.title('SIP Scale')
    plt.plot(scales)
    plt.ylabel('Value')
    plt.xlabel('Months')
    plt.show()
    

## Function of calculating rising or falling

In [28]:
def monthFilter(month, length, data):
    subData = np.array(data[(month - (month % 6)) : (month + (5-(month % 6)) + 1)])
    value=(data[month] - np.mean(subData)) / np.std(subData)
    if(value> 1 or value < -1):
        return value
    else:
        return 0

## This function will get various people,aliveList,variations

In [64]:
def preprocessing():
    id_list = []
    id_temp = []
    aliveList=[]
    peopleChinaList = []
    peopleIndiaList = []
    peopleBrazilList = []
    peopleRussiaList = []
    peopleMedicalList = []
    markMonthChinaList =[]
    markMonthIndiaList = []
    markMonthBrazilList = []
    markMonthRussiaList = []
    markMonthlMedicalList = []
    MonthChinaValue=[]
    MonthIndiaValue=[]
    MonthRussiaValue=[]
    MonthBrazilValue=[]
    for df in df_corrects:
        for filter_id in df['基金統編']:
#             id = str(filter_id)
#             id = id.zfill(8)
            id_temp.append(filter_id)
    id_list = list(set(id_temp))
    for id in id_list:
            isAlive = True
            navs=[]
            money=[]
            num=[]
            people=[]
            scales=[]
            name = ''
            isSaveList = []
            for i in range((17-3)*12 + 4,17*12 + 4):
                try:
                    name = df_corrects[i][df_corrects[i]['基金統編']==id]['基金名稱'].values[0]
                    navs.append(df_corrects[i][df_corrects[i]['基金統編']==id]['單位淨值'].values[0])
                    money.append(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款金額'].values[0])
                    num.append(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款筆數'].values[0])
                    people.append(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款人數'].values[0])
                    scales.append(df_corrects[i][df_corrects[i]['基金統編']==id]['基金規模'].values[0])
                    if(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款人數'].values[0]) == 0 :
                        isAlive = False
                    if(i == 17*12 + 3 and isAlive):
                        aliveList.append(df_corrects[i][df_corrects[i]['基金統編']==id]['基金名稱'].values[0])
                except:
                    isAlive = False
                    navs.append(0)
                    money.append(0)
                    num.append(0)
                    people.append(0)
                    scales.append(0)
                    break
            if(isAlive):
                for i in range(0, len(people)):
                    if("中國" in name):
                        try:
                            peopleChinaList[i] += people[i]
                        except:
                            peopleChinaList.append(people[i])
                    elif("印度" in name):
                        try:
                            peopleIndiaList[i] += people[i]
                        except:
                            peopleIndiaList.append(people[i])
                    elif("巴西" in name):
                        try:
                            peopleBrazilList[i] += people[i]
                        except:
                            peopleBrazilList.append(people[i])
                    elif("俄羅斯" in name):
                        try:
                            peopleRussiaList[i] += people[i]
                        except:
                            peopleRussiaList.append(people[i])
                    else:
                        try:
                            peopleMedicalList[i] += people[i] 
                        except:
                            peopleMedicalList.append(people[i])
#                 plt.plot(navs)
#                 plt.title(id + 10*' ' +'Unit net value')
#                 plt.ylabel('Value')
#                 plt.xlabel('Months')
#                 plt.show()
#                 plt.title(id + 10*' ' +'SIP value')
#                 plt.plot(money)
#                 plt.ylabel('Value')
#                 plt.xlabel('Months')
#                 plt.show()
#                 plt.title(id + 10*' ' +'SIP Number of records')
#                 plt.plot(num)
#                 plt.ylabel('Value')
#                 plt.xlabel('Months')
#                 plt.show()
#                 plt.title(id + 10*' ' +'SIP Number of people')
#                 plt.plot(people)
#                 plt.ylabel('Value')
#                 plt.xlabel('Months')
#                 plt.show()
#                 plt.title(id + 10*' ' +'SIP Scale')
#                 plt.plot(scales)
#                 plt.ylabel('Value')
#                 plt.xlabel('Months')
#                 plt.show()

    # Calculate people and value
    
    china={}
    for i in range(0, len(peopleChinaList)):
        value=monthFilter(i, 6, peopleChinaList)
        if value!=0:
            markMonthChinaList.append(i)
            MonthChinaValue.append(value)
    china={
        'Month_Index':markMonthChinaList,
        'Value':MonthChinaValue
    }
    brazil={}
    for i in range(0, len(peopleBrazilList)):
        value=monthFilter(i, 6, peopleBrazilList)
        if value!=0:
            markMonthBrazilList.append(i)
            MonthBrazilValue.append(value)
    brazil={
        'Month_Index':markMonthBrazilList,
        'Value':MonthBrazilValue
    }
    for i in range(0, len(peopleIndiaList)):
        value=monthFilter(i, 6, peopleIndiaList)
        if value!=0:
            markMonthIndiaList.append(i)
            MonthIndiaValue.append(value)
    india={
        'Month_Index':markMonthBrazilList,
        'Value':MonthBrazilValue
    }
    for i in range(0, len(peopleRussiaList)):
        value=monthFilter(i, 6, peopleIndiaList)
        if value!=0:
            markMonthIndiaList.append(i)
            MonthIndiaValue.append(value)
    russia={
        'Month_Index':markMonthRussiaList,
        'Value':MonthRussiaValue
    }
    variation={
        'China':pd.DataFrame(china),
        'Brazil':pd.DataFrame(brazil),
        'India':pd.DataFrame(india),
        'Russia':pd.DataFrame(russia)
    }
    
#     for i in range(0, len(peopleMedicalList)):
#         if(monthFilter(i, 6, peopleMedicalList)):
#             markMonthlMedicalList.append(i)
#     plt.title('SIP Number of people in Medical')
#     plt.plot(peopleMedicalList)
#     plt.ylabel('Value')
#     plt.xlabel('Months')
#     plt.show()
    china_df=pd.DataFrame(china)
    dict={}
    start_date = "2016/04" 
    stop_date="2019/03"
    start=datetime.strptime(start_date,"%Y/%m")
    stop=datetime.strptime(stop_date,"%Y/%m")
    arr_smonth=[]
    while start<=stop:
        arr_smonth.append(start.strftime('%Y/%m')) # Return a string representing the date and time
        start = start +  relativedelta(months=1)
    percentage_c=[]
    percentage_b=[]
    percentage_r=[]
    percentage_i=[]
    for i in range(len(peopleChinaList)):
        total=peopleChinaList[i]+peopleIndiaList[i]+peopleRussiaList[i]+peopleBrazilList[i]
        percentage_c.append((peopleChinaList[i]/total)*100)
        percentage_b.append((peopleBrazilList[i]/total)*100)
        percentage_r.append((peopleRussiaList[i]/total)*100)
        percentage_i.append((peopleIndiaList[i]/total)*100)
        
        
    
    dict={
        'Date':arr_smonth,
        'China':peopleChinaList,
        'India':peopleIndiaList,
        'Russia':peopleRussiaList,
        'Brazil':peopleBrazilList,
        'China%':percentage_c,
        'India%':percentage_i,
        'Russia%':percentage_r,
        'Brazil%':percentage_b
    }
 
    peopleDF=pd.DataFrame(dict)
    return aliveList,variation,peopleDF



In [65]:
aliveList,varation ,people= preprocessing()
varation['China']

,Month_Index,Value
0,0,-1.856616
1,9,-2.054153
2,12,1.923390
3,17,-1.151120
4,20,-1.889533
5,22,1.122276
6,24,-1.470169
7,29,1.457625
8,33,-1.806295


In [66]:
people

,Date,China,India,Russia,Brazil,China%,India%,Russia%,Brazil%
0,2016/04,19451.0,2267.0,2104.0,3244.0,71.865071,8.375822,7.773590,11.985517
1,2016/05,22223.0,2247.0,2058.0,3210.0,74.729303,7.555989,6.920438,10.794270
2,2016/06,22482.0,2231.0,2040.0,3153.0,75.175550,7.460041,6.821374,10.543035
3,2016/07,22306.0,2442.0,1965.0,2971.0,75.144859,8.226654,6.619728,10.008759
4,2016/08,20813.0,2413.0,1881.0,2754.0,74.702990,8.660852,6.751373,9.884785
5,2016/09,21304.0,2360.0,1811.0,2597.0,75.890567,8.406954,6.451268,9.251211
6,2016/10,21493.0,2390.0,1738.0,2490.0,76.457614,8.502010,6.182633,8.857743
7,2016/11,21451.0,2314.0,1710.0,2375.0,77.023339,8.308797,6.140036,8.527828
8,2016/12,20909.0,2231.0,1663.0,2222.0,77.369103,8.255319,6.153562,8.222017
9,2017/01,17905.0,2046.0,1535.0,2062.0,76.036181,8.688636,6.518600,8.756582


In [67]:
aliveList

['第一金中國世紀基金-新臺幣',
 '摩根中國亮點基金',
 '群益全球關鍵生技基金-新臺幣',
 '日盛中國內需動力基金',
 '瀚亞巴西基金',
 '群益中國新機會基金-新臺幣',
 '台新中國精選中小基金',
 '野村中國機會基金',
 '兆豐國際中國A股基金(台幣)',
 '野村全球生技醫療基金',
 '保德信全球醫療生化基金-新臺幣',
 '元大印度基金',
 '野村新興傘型基金之大俄羅斯基金',
 '野村雙印傘型基金之印度潛力基金',
 '安聯全球生技趨勢基金-新臺幣',
 '野村巴西基金',
 '安聯中國策略基金',
 '國泰中國內需增長基金台幣級別',
 '復華中國新經濟A股基金-新臺幣',
 '德信中國精選成長基金',
 '台新印度基金',
 '兆豐國際中國A股基金(美金)',
 '安聯中國東協基金']

In [68]:
newAliveList = []
for name in aliveList:
    if("新臺幣" in name):
        name = name.replace("新臺幣", "台幣")
    if("新台幣" in name):
        name = name.replace("新台幣", "台幣")
    if("台幣類型" in name):
        name = name.replace("台幣類型", "台幣")
    if("N" in name):
        name = name.replace("N", "N類型")
    if("-N類型" in name):
        name = name.replace("-N類型", "")
        name = name.replace("基金", "基金N類型")
    if("級別" in name):
        if("台幣" in name):
            name = name.replace("台幣級別", "(台幣)")
        if("人民幣" in name):
            name = name.replace("人民幣級別", "(人民幣)")
        if("美元" in name):
            name = name.replace("美元級別", "(美元)")
    if("-" in name):
        name = name.replace("-", "(", 1) + ")"
    if("人民幣類型" in name):
        name = name.replace("類型", "")
    if("兆豐國際中國A股基金(美金)" in name):
        name = name.replace("美金", "美元")
    if("安聯中國策略基金" in name):
        name = name + "(台幣)"
    if("台新中國精選中小基金" in name):
        name = name + "(台幣)"
    if("摩根中國A股基金" in name):
        if("摩根中國A股基金(美元)" in name):
            pass
        else:
            name = name + "(台幣)"
    if("瀚亞中國" in name):
        name = name.replace("中國", "中國A股")
    if("野村新興傘型基金之大俄羅斯基金" in name):
        name = name.replace("野村新興傘型基金之大俄羅斯基金", "野村大俄羅斯基金")
    if("野村雙印傘型基金之印度潛力基金" in name):
        name = name.replace("野村雙印傘型基金之印度潛力基金", "野村印度潛力基金")
    newAliveList.append(name)
# newAliveList

In [69]:
newAliveList

['第一金中國世紀基金(台幣)',
 '摩根中國亮點基金',
 '群益全球關鍵生技基金(台幣)',
 '日盛中國內需動力基金',
 '瀚亞巴西基金',
 '群益中國新機會基金(台幣)',
 '台新中國精選中小基金(台幣)',
 '野村中國機會基金',
 '兆豐國際中國A股基金(台幣)',
 '野村全球生技醫療基金',
 '保德信全球醫療生化基金(台幣)',
 '元大印度基金',
 '野村大俄羅斯基金',
 '野村印度潛力基金',
 '安聯全球生技趨勢基金(台幣)',
 '野村巴西基金',
 '安聯中國策略基金(台幣)',
 '國泰中國內需增長基金(台幣)',
 '復華中國新經濟A股基金(台幣)',
 '德信中國精選成長基金',
 '台新印度基金',
 '兆豐國際中國A股基金(美元)',
 '安聯中國東協基金']

In [70]:
pd.set_option('display.max_rows', None)
df2 = pd.read_html("https://www.moneydj.com/funddj/yb/YP302000.djhtm?a=ET003001")
df3 = pd.read_html("https://www.moneydj.com/funddj/yb/yp302000.djhtm?a=ET003002")
df4 = pd.read_html("https://www.moneydj.com/funddj/yb/yp302000.djhtm?a=ET003003")
dfFinal = pd.concat([df2[4], df3[4], df4[4]], axis=0)
dfFinal

,日期,基金名稱,基金公司,淨值,幣別,報酬率(%),風險,Unnamed: 7_level_0,Unnamed: 8_level_0,Unnamed: 9_level_0,Unnamed: 10_level_0
,三個月,六個月,一年,三年,標準差,Beta,Unnamed: 6_level_1,Unnamed: 7_level_1,Unnamed: 8_level_1,Unnamed: 9_level_1,Unnamed: 10_level_1
0,06/05,第一金全球AI FinTech金融科技基金(台幣),第一金投信,14.8200,台幣,7.55,19.42,10.76,NaN,17.00,1.1382
1,06/05,第一金全球AI FinTech金融科技基金N類型(台幣),第一金投信,14.8300,台幣,7.54,19.40,10.75,NaN,17.03,1.1388
2,06/05,第一金全球AI FinTech金融科技基金(美元),第一金投信,15.1623,美元,5.68,16.80,5.19,NaN,17.64,1.2246
3,06/05,第一金全球AI FinTech金融科技基金N類型(美元),第一金投信,15.1620,美元,5.68,16.79,5.19,NaN,17.64,1.2249
4,06/05,永豐主流品牌基金,永豐投信,18.7900,台幣,5.15,17.00,2.34,49.60,13.45,0.8499
5,06/05,中國信託全球股票入息基金-分配型(本基金之配息來源可能為本金),中國信託投信,9.0000,台幣,4.09,12.37,-4.55,1.61,18.25,1.1218
6,06/05,中國信託全球股票入息基金-累積型(本基金之配息來源可能為本金),中國信託投信,9.7400,台幣,4.06,12.34,-4.51,1.67,18.20,1.1142
7,06/05,野村環球基金(台幣),野村投信,18.7400,台幣,5.58,11.81,9.59,42.29,13.29,0.9313
8,06/05,台新智慧生活基金(台幣),台新投信,10.7200,台幣,2.19,10.74,2.98,NaN,16.64,0.8847


In [71]:
for name in newAliveList:
    filterName = (dfFinal["基金名稱"]["基金名稱"] == name)
    try:
        print(dfFinal[filterName]["報酬率(%)"]["六個月"].values[0])
    except:
        print(name)

KeyError: '基金名稱'